In [44]:
import pandas as pd

df = pd.read_csv('Informe Listado de mesas.csv')

Modificación de nombres para mejorar la búsqueda, los resultados pueden verse en listaescuelas.csv:

In [45]:
import re

def limpiar_nombre_escuela(nombre):
    # Reemplaza abreviaturas comunes y símbolos con palabras completas
    nombre = re.sub(r'\bESC\b', 'Escuela', nombre)
    nombre = re.sub(r'\bCOL\b', 'Colegio', nombre)
    nombre = re.sub(r'\bINST\b', 'Instituto', nombre)
    nombre = re.sub(r'\bPOLIM\b', 'Polimodal', nombre)
    nombre = re.sub(r'\bGDOR\b', 'Gobernador', nombre)
    nombre = re.sub(r'\bDR\b', 'Doctor', nombre)
    nombre = re.sub(r'\bGRAL\b', 'General', nombre)
    nombre = re.sub(r'\bTTE\b', 'Teniente', nombre)
    nombre = re.sub(r'\bPCIA\b', 'Provincia', nombre)
    
    # Eliminar repeticiones de "Número"
    nombre = re.sub(r'Número\s+', 'Número ', nombre)
    
    # Elimina comillas y otros caracteres especiales innecesarios
    nombre = re.sub(r'""', '', nombre)
    nombre = re.sub(r'[^\w\s]', '', nombre)

    # Eliminar espacios múltiples
    nombre = re.sub(r'\s+', ' ', nombre).strip()
    
    return nombre


Ejecución de la función y guardado de archivo:

In [46]:
df['nombre_limpio'] = df['nombre'].apply(limpiar_nombre_escuela)

ListaEscuelas = df[['nombre', 'nombre_limpio']]

In [47]:
ListaEscuelas.to_csv("listaescuelas.csv")

Ejecución de la búsqueda de coordenadas:

In [48]:
import googlemaps
import os

API_KEY = os.environ.get("API_KEY")
gmaps = googlemaps.Client(key=API_KEY)

CoordenadasEscuelas = []

# Función para obtener coordenadas
def obtener_coordenadas(nombre_escuela):
    try:
        consulta = f'"{nombre_escuela}", Provincia de Corrientes, Argentina'
        # Geocodificar el nombre de la escuela
        geocode_result = gmaps.geocode(
            consulta,
            region='ar'  # Limitar la búsqueda a Argentina
        )
        if geocode_result:
            # Obtener las coordenadas del primer resultado
            coordenadas = geocode_result[0]['geometry']['location']
            return (coordenadas['lat'], coordenadas['lng'])
        else:
            return None
    except Exception as e:
        print(f"Error al obtener coordenadas para {nombre_escuela}: {e}")
        return None

for _, row in ListaEscuelas.iterrows():
    nombre_original = row["nombre"]
    nombre_limpio = row["nombre_limpio"]
    
    # Usar el nombre limpio para buscar las coordenadas
    coordenadas = obtener_coordenadas(nombre_limpio)

    if coordenadas:
        print(f"Coordenadas de {nombre_original}: Latitud {coordenadas[0]}, Longitud {coordenadas[1]}")
        # Almacenar el nombre original y las coordenadas en la lista
        CoordenadasEscuelas.append((nombre_original, coordenadas[0], coordenadas[1]))
    else:
        print(f"No se encontraron coordenadas para {nombre_original}.")

Coordenadas de ESC- Nº 3 "DEL CENTENARIO": Latitud -27.470343, Longitud -58.8399214
Coordenadas de ESC- Nº 2 "DOMINGO F- SARMIENTO": Latitud -28.5841599, Longitud -58.00719220000001
Coordenadas de ESC- Nº 10 "R- DE ESCALADA DE SAN MARTÍN": Latitud -30.0895819, Longitud -58.7821958
Coordenadas de ESC- Nº 402 "GENARO BERÓN DE ASTRADA": Latitud -27.4836397, Longitud -58.8455869
Coordenadas de COLEGIO POLIMODAL "IBERÁ": Latitud -28.5841599, Longitud -58.00719220000001
Coordenadas de ESC- Nº 9 "GUSTAVO SOLANO GÓMEZ": Latitud -27.4766082, Longitud -58.84668060000001
Coordenadas de ESC- Nº 11 "DR- LISANDRO SEGOVIA": Latitud -27.4831936, Longitud -58.836124
Coordenadas de ESC- Nº 290 "DR- JUAN R- VIDAL": Latitud -27.3702201, Longitud -58.5112662
Coordenadas de ESC- Nº 850 "GDOR- JOSÉ M- DE LLANO": Latitud -28.5841599, Longitud -58.00719220000001
Coordenadas de ESC- Nº 291 "DR- EUDORO VARGAS GÓMEZ": Latitud -28.5841599, Longitud -58.00719220000001
Coordenadas de ESC- Nº 299 "JUAN ALFREDO ASUAD"

Obtención de las coordenadas de "Provincia de Corrientes" (las escuelas que no daban resultados devuelven estas mismas coordenadas), estas son tenidas en cuenta para eliminar del mapa de calor los resultados genéricos:

In [51]:
obtener_coordenadas("")

(-28.5841599, -58.00719220000001)

Guardado de archivo:

In [49]:
import pandas as pd

df = pd.DataFrame(CoordenadasEscuelas)

# Asignar nombres a las columnas
df.columns = ['Escuela', 'Latitud', 'Longitud']

# Guardar el DataFrame como un archivo CSV
df.to_csv("Coordenadas.csv", index=False, encoding='utf-8')